In [64]:
from bs4 import BeautifulSoup
import urllib 
import re
from urllib.parse import unquote
import string
import dateutil.parser as dateparser
from time import sleep
from pprint import pprint
import os
import json
import datetime


folder = os.path.join('C:/', 'users', 'charr', 'documents', 'soundtrack')
letters = list(string.ascii_uppercase)

In [14]:
movie_list = []
for letter in letters:
    html_text = urllib.request.urlopen(f"http://www.soundtrackinfo.com/lettersearch/{letter}/").read()
    soup = BeautifulSoup(html_text, 'html.parser')

    letter_list = str(soup.find_all(['td', 'tr'])[80:]).split("\n")[1:-1]

    index_list = []
    for string in letter_list:
        lst = [i for i,x in enumerate(string) if x == '>' or x == '<' ]
        index_list.append(lst)

    
    for i,x in enumerate(letter_list):
        movie_list.append([x[index_list[i][1]+1:index_list[i][2]], x[index_list[i][3]+1:index_list[i][4]]])
    
#movie_list

In [15]:
len(movie_list)

13973

In [16]:
url_list = []
for letter in letters:
    html_text = urllib.request.urlopen(f"http://www.soundtrackinfo.com/lettersearch/{letter}/").read()
    soup = BeautifulSoup(html_text, 'html.parser')

    for i in soup.find_all(['td', 'tr', 'a'])[80:]:
        if i.get('href') != None:
            url_list.append(i.get('href'))
            
#url_list

In [17]:
len(url_list)

12457

In [60]:
def save_tweet_dict(dct):
    file_suffix = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = 'soundtrack_data' + file_suffix + '.json'
    filepath = os.path.join(folder, filename)
    with open(filepath, mode='w', encoding = 'utf-8') as f:
        json.dump(dct, f)

In [66]:
soundtrack_dct = {}
error_list = []
for url in url_list:
    if len(soundtrack_dct) > 1000:
        save_tweet_dict(soundtrack_dct)
        soundtrack_dct = {}
    else: 
        try:
            if "/lettersearch/only" not in url:
                html_text = urllib.request.urlopen(f"http://www.soundtrackinfo.com/tracks{url[4:]}").read()
                soup = BeautifulSoup(html_text, 'html.parser')

                tables = [[[td.get_text(strip=True) for td in tr.find_all('td')] for tr in table.find_all('tr')] for table in soup.find_all('table')]
                soundtrack_dct[url[4:].strip('/')] = tables[2][0]
        except:
            error_list.append(url)


In [67]:
#final soundtrack_dct for last group of <1000 movies
save_tweet_dict(soundtrack_dct)

In [68]:
len(error_list)

0